# Chapter 27: ProcessPoolExecutor and Futures

The `concurrent.futures` module provides a high-level interface for parallel execution.
`ProcessPoolExecutor` manages a pool of worker processes and returns `Future` objects
that represent pending results. It is simpler and more Pythonic than the lower-level
`multiprocessing.Pool` API.

## Topics Covered
- **ProcessPoolExecutor**: High-level process pool interface
- **Future**: Representing pending results
- **map() vs submit()**: Two ways to distribute work
- **as_completed()**: Processing results as they become available
- **CPU-bound vs I/O-bound**: Choosing the right executor
- **Practical**: Comparing ProcessPoolExecutor and ThreadPoolExecutor

## ProcessPoolExecutor Basics

`ProcessPoolExecutor` is part of `concurrent.futures` and provides the same API as
`ThreadPoolExecutor`. It uses processes instead of threads, making it ideal for
**CPU-bound** workloads.

Key features:
- Context manager support (`with` statement)
- `max_workers` defaults to the number of CPUs
- Worker functions must be **picklable** (defined at module level)
- Returns `Future` objects for asynchronous result handling

In [ ]:
import os
from concurrent.futures import ProcessPoolExecutor


def square(x: int) -> int:
    """Compute the square of a number."""
    return x * x


# Basic usage with executor.map()
print(f"CPU count: {os.cpu_count()}")

with ProcessPoolExecutor(max_workers=2) as executor:
    results: list[int] = list(executor.map(square, [1, 2, 3, 4, 5]))

print(f"Squares: {results}")

## executor.map(): Ordered Parallel Mapping

`executor.map(func, *iterables)` applies a function to every item in the iterables,
distributing work across the pool. It returns an **iterator** that yields results in
the **same order** as the input.

Key differences from `multiprocessing.Pool.map()`:
- Returns a lazy iterator (not a list)
- Supports a `timeout` parameter
- Supports a `chunksize` parameter for efficiency with large iterables

In [ ]:
import os
from concurrent.futures import ProcessPoolExecutor


def process_item(x: int) -> tuple[int, int, int]:
    """Process an item and return (input, result, pid)."""
    return (x, x * x, os.getpid())


# map() preserves input order
with ProcessPoolExecutor(max_workers=2) as executor:
    results = list(executor.map(process_item, range(8)))

main_pid: int = os.getpid()
print(f"Main PID: {main_pid}")
print("\nResults (order preserved):")
for x, squared, pid in results:
    print(f"  {x}^2 = {squared} (worker PID: {pid})")

worker_pids: set[int] = {pid for _, _, pid in results}
print(f"\nUnique worker PIDs: {worker_pids}")
print(f"All workers differ from main: {all(pid != main_pid for _, _, pid in results)}")

## executor.submit() and Future Objects

`executor.submit(func, *args, **kwargs)` submits a single callable for execution and
returns a `Future` object immediately. The `Future` represents a computation that may
not have completed yet.

Key `Future` methods:
- `result(timeout=None)` -- block until the result is ready, then return it
- `done()` -- return `True` if the computation has completed
- `cancelled()` -- return `True` if the task was cancelled
- `exception(timeout=None)` -- return the exception, if any
- `add_done_callback(fn)` -- register a callback to run when the future completes

In [ ]:
from concurrent.futures import Future, ProcessPoolExecutor


def square(x: int) -> int:
    """Compute the square of a number."""
    return x * x


# submit() returns a Future
with ProcessPoolExecutor(max_workers=1) as executor:
    future: Future[int] = executor.submit(square, 7)

    # Inspect the future
    print(f"Type: {type(future).__name__}")
    print(f"Done: {future.done()}")  # May or may not be done yet

    # Get the result (blocks if not yet ready)
    result: int = future.result()
    print(f"Result: {result}")
    print(f"Done after result(): {future.done()}")
    print(f"Exception: {future.exception()}")

In [ ]:
from concurrent.futures import Future, ProcessPoolExecutor


def cube(x: int) -> int:
    """Compute the cube of a number."""
    return x ** 3


# Submit multiple tasks and collect futures
with ProcessPoolExecutor(max_workers=2) as executor:
    futures: dict[Future[int], int] = {
        executor.submit(cube, n): n
        for n in range(1, 7)
    }

    # Retrieve results from each future
    print("Results from submit():")
    for future, input_val in futures.items():
        result: int = future.result()
        print(f"  {input_val}^3 = {result}")

## map() vs submit(): When to Use Each

| Feature | `map()` | `submit()` |
|---------|---------|:-----------|
| Returns | Iterator of results | Individual `Future` objects |
| Order | Results in input order | Results in completion order (with `as_completed`) |
| Arguments | Single iterable per param | Full control of args/kwargs |
| Use case | Batch processing same operation | Different tasks or custom handling |
| Error handling | Exception raised during iteration | Exception stored in `Future` |

**Rule of thumb**: Use `map()` for simple "apply same function to many inputs" tasks.
Use `submit()` when you need more control over individual tasks.

## as_completed(): Results in Completion Order

`concurrent.futures.as_completed(futures)` yields futures as they **finish**, not in
the order they were submitted. This is useful when you want to process results as
soon as they are available, rather than waiting for all of them.

In [ ]:
import time
from concurrent.futures import Future, ProcessPoolExecutor, as_completed


def variable_work(task_id: int) -> tuple[int, float]:
    """Simulate work with variable duration."""
    # Tasks with higher IDs finish faster
    duration: float = 0.5 - (task_id * 0.1)
    duration = max(duration, 0.05)
    time.sleep(duration)
    return (task_id, duration)


with ProcessPoolExecutor(max_workers=3) as executor:
    futures: dict[Future[tuple[int, float]], int] = {
        executor.submit(variable_work, i): i
        for i in range(5)
    }

    print("Results in completion order:")
    for future in as_completed(futures):
        task_id, duration = future.result()
        print(f"  Task {task_id} completed (took {duration:.2f}s)")

## Error Handling with Futures

When a worker function raises an exception, it is captured by the `Future` object and
re-raised when you call `.result()` or `.exception()`. This makes error handling
straightforward.

In [ ]:
from concurrent.futures import Future, ProcessPoolExecutor


def divide(a: float, b: float) -> float:
    """Divide a by b. Raises ZeroDivisionError if b is zero."""
    return a / b


with ProcessPoolExecutor(max_workers=2) as executor:
    # Submit tasks, one of which will fail
    tasks: list[tuple[float, float]] = [(10, 2), (20, 4), (30, 0), (40, 5)]
    futures: list[tuple[Future[float], tuple[float, float]]] = [
        (executor.submit(divide, a, b), (a, b))
        for a, b in tasks
    ]

    for future, (a, b) in futures:
        try:
            result: float = future.result()
            print(f"  {a} / {b} = {result}")
        except ZeroDivisionError:
            print(f"  {a} / {b} = ERROR (division by zero)")
            # Inspect the exception stored in the Future
            print(f"    future.exception() = {future.exception()!r}")

## CPU-Bound vs I/O-Bound Workloads

Choosing between `ProcessPoolExecutor` and `ThreadPoolExecutor` depends on the nature
of your workload:

| Workload | Bottleneck | Best Executor | Why |
|----------|-----------|:--------------|:----|
| **CPU-bound** | Computation | `ProcessPoolExecutor` | Bypasses GIL, true parallelism |
| **I/O-bound** | Network, disk | `ThreadPoolExecutor` | Threads release GIL during I/O |
| **Mixed** | Both | Depends | Profile first; consider async I/O |

The GIL (Global Interpreter Lock) prevents multiple threads from executing Python bytecode
simultaneously, but it is released during I/O operations. Processes each have their own GIL,
so CPU-bound work runs truly in parallel.

In [ ]:
import time
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor


def cpu_heavy(n: int) -> int:
    """CPU-bound: compute sum of squares."""
    total: int = 0
    for i in range(n):
        total += i * i
    return total


workload: list[int] = [1_500_000] * 8

# Sequential baseline
start: float = time.perf_counter()
_ = [cpu_heavy(n) for n in workload]
seq_time: float = time.perf_counter() - start

# Threads (limited by GIL for CPU-bound work)
start = time.perf_counter()
with ThreadPoolExecutor(max_workers=4) as executor:
    _ = list(executor.map(cpu_heavy, workload))
thread_time: float = time.perf_counter() - start

# Processes (bypasses GIL)
start = time.perf_counter()
with ProcessPoolExecutor(max_workers=4) as executor:
    _ = list(executor.map(cpu_heavy, workload))
process_time: float = time.perf_counter() - start

print("CPU-bound workload comparison:")
print(f"  Sequential:           {seq_time:.3f}s")
print(f"  ThreadPoolExecutor:   {thread_time:.3f}s (speedup: {seq_time/thread_time:.2f}x)")
print(f"  ProcessPoolExecutor:  {process_time:.3f}s (speedup: {seq_time/process_time:.2f}x)")
print(f"\nProcessPoolExecutor wins for CPU-bound tasks!")

In [ ]:
import time
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor


def io_heavy(seconds: float) -> float:
    """I/O-bound: simulate waiting for a network response."""
    time.sleep(seconds)
    return seconds


io_workload: list[float] = [0.2] * 8

# Sequential baseline
start: float = time.perf_counter()
_ = [io_heavy(s) for s in io_workload]
seq_time: float = time.perf_counter() - start

# Threads (efficient for I/O)
start = time.perf_counter()
with ThreadPoolExecutor(max_workers=4) as executor:
    _ = list(executor.map(io_heavy, io_workload))
thread_time: float = time.perf_counter() - start

# Processes (more overhead for I/O)
start = time.perf_counter()
with ProcessPoolExecutor(max_workers=4) as executor:
    _ = list(executor.map(io_heavy, io_workload))
process_time: float = time.perf_counter() - start

print("I/O-bound workload comparison:")
print(f"  Sequential:           {seq_time:.3f}s")
print(f"  ThreadPoolExecutor:   {thread_time:.3f}s (speedup: {seq_time/thread_time:.2f}x)")
print(f"  ProcessPoolExecutor:  {process_time:.3f}s (speedup: {seq_time/process_time:.2f}x)")
print(f"\nBoth work for I/O, but threads have less overhead!")

## Callbacks on Futures

You can attach callbacks to `Future` objects using `add_done_callback()`. The callback
function is called with the `Future` as its argument when the task completes. This is
useful for logging, chaining tasks, or updating progress.

In [ ]:
from concurrent.futures import Future, ProcessPoolExecutor


def compute_factorial(n: int) -> int:
    """Compute n! iteratively."""
    result: int = 1
    for i in range(2, n + 1):
        result *= i
    return result


def on_complete(future: Future[int]) -> None:
    """Callback when a factorial computation completes."""
    result: int = future.result()
    print(f"  Callback: computation finished, result = {result}")


with ProcessPoolExecutor(max_workers=2) as executor:
    print("Submitting tasks with callbacks:")
    for n in [5, 8, 10, 12]:
        future: Future[int] = executor.submit(compute_factorial, n)
        future.add_done_callback(on_complete)

print("All tasks completed.")

## os.cpu_count() and Choosing max_workers

`os.cpu_count()` returns the number of logical CPUs on the system. This is the default
value for `max_workers` in `ProcessPoolExecutor`.

Guidelines for choosing `max_workers`:
- **CPU-bound**: Use `os.cpu_count()` or slightly less (leave room for other processes)
- **I/O-bound**: Can use more workers than CPUs since they spend time waiting
- **Memory-constrained**: Each process uses its own memory, so watch usage

In [ ]:
import os
from concurrent.futures import ProcessPoolExecutor


def get_pid(_: int = 0) -> int:
    """Return the current process ID."""
    return os.getpid()


# os.cpu_count() tells us how many CPUs are available
cpus: int | None = os.cpu_count()
print(f"os.cpu_count(): {cpus}")
assert cpus is not None
assert cpus >= 1

# Default max_workers = cpu_count
with ProcessPoolExecutor() as executor:
    # _max_workers is an implementation detail, shown for educational purposes
    actual_workers: int | None = executor._max_workers
    print(f"Default max_workers: {actual_workers}")

# Practical: verify processes run on different PIDs
with ProcessPoolExecutor(max_workers=4) as executor:
    pids: list[int] = list(executor.map(get_pid, range(8)))
    unique: set[int] = set(pids)
    print(f"\n8 tasks across 4 workers:")
    print(f"  PIDs: {pids}")
    print(f"  Unique PIDs: {unique}")
    print(f"  All differ from main ({os.getpid()}): {all(p != os.getpid() for p in pids)}")

## Practical: Real-World Pattern -- Parallel Data Processing

A common real-world pattern is to process a batch of items in parallel, collecting
successes and failures separately. This example demonstrates using `submit()` with
error handling for robust parallel processing.

In [ ]:
from concurrent.futures import Future, ProcessPoolExecutor, as_completed


def process_record(record_id: int) -> dict[str, int | str]:
    """Simulate processing a data record. Some records fail."""
    if record_id % 5 == 0:
        raise ValueError(f"Record {record_id} is corrupted")
    return {"id": record_id, "status": "processed", "value": record_id * 10}


record_ids: list[int] = list(range(1, 16))
successes: list[dict[str, int | str]] = []
failures: list[tuple[int, str]] = []

with ProcessPoolExecutor(max_workers=3) as executor:
    # Submit all tasks
    future_to_id: dict[Future[dict[str, int | str]], int] = {
        executor.submit(process_record, rid): rid
        for rid in record_ids
    }

    # Process results as they complete
    for future in as_completed(future_to_id):
        record_id: int = future_to_id[future]
        try:
            result: dict[str, int | str] = future.result()
            successes.append(result)
        except ValueError as e:
            failures.append((record_id, str(e)))

print(f"Processed {len(successes)} records successfully:")
for s in sorted(successes, key=lambda x: x["id"]):
    print(f"  Record {s['id']}: value={s['value']}")

print(f"\n{len(failures)} records failed:")
for record_id, error in sorted(failures):
    print(f"  Record {record_id}: {error}")

## Summary

### Key Takeaways

| Concept | API | Purpose |
|---------|-----|:--------|
| **Executor** | `ProcessPoolExecutor(max_workers=n)` | High-level process pool |
| **map()** | `executor.map(func, iterable)` | Parallel map, results in input order |
| **submit()** | `executor.submit(func, *args)` | Submit single task, returns Future |
| **Future** | `future.result()`, `.done()`, `.exception()` | Pending result handle |
| **as_completed()** | `as_completed(futures)` | Yield futures in completion order |
| **Callbacks** | `future.add_done_callback(fn)` | Run code when a future completes |
| **cpu_count()** | `os.cpu_count()` | Number of logical CPUs |

### Choosing the Right Executor

| Scenario | Use |
|----------|:----|
| Heavy computation (math, parsing, compression) | `ProcessPoolExecutor` |
| Network requests, file I/O, database queries | `ThreadPoolExecutor` |
| Simple batch: same function, many inputs | `executor.map()` |
| Complex: different tasks, error handling, callbacks | `executor.submit()` |

### Best Practices
- Always use executors as context managers to ensure proper cleanup
- Define worker functions at module level (they must be picklable)
- Use `as_completed()` when you want to process results as soon as they are ready
- Handle exceptions from `.result()` -- they are re-raised from the worker
- For CPU-bound tasks, `max_workers=os.cpu_count()` is a good starting point
- Prefer `ProcessPoolExecutor` over `multiprocessing.Pool` for new code -- it has a cleaner API